# Workshop 2: CASE_WHEN, VIEWs und User Defined Functions (UDF)

In diesem Workshop geht es um Techniken, um häufig benutzte Queries zu automatisieren und die zu optimieren. 

* Schwierigkeit: einfach
* Prüfungsrelevant: ja (ausser UDF)
* Zeitaufwand: 30min



In [2]:
# Hilfsfunktionen laden und ...
from util.sqlite_util import *

# ... Test-Datenbank öffnen
path = get_connection_path("chinook.sqlite")
connection = create_connection(path)

Connection to SQLite DB 2.6.0 (./data/chinook.sqlite) successful


## 1. CASE WHEN

* CASE WHEN ist vergleichbar mit if-then-else in Python
* CASE WHEN kann an beliebigen Stellen in einer Query verwendet werden. 
* Üblich ist der SELECT-Part, um Daten zu formatieren (siehe Beispiel)
* Aufgabe 1 zeigt eine Alternative in der ORDER BY Clause. 

``` sql
CASE case_expression
     WHEN when_expression_1 THEN result_1
     WHEN when_expression_2 THEN result_2
     ...
     [ ELSE result_else ] 
END [field_name]
```

In [3]:
# Beispiel 1.1
stmt = """SELECT
        trackid, name,
    CASE
        WHEN milliseconds <= 60000 THEN 'short'
        WHEN milliseconds > 60000 AND 
            milliseconds < 300000 THEN 'medium'
        ELSE
            'long'
        END category
FROM
    tracks
LIMIT 10;
"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'For Those About To Rock (We Salute You)', 'long')
(2, 'Balls to the Wall', 'long')
(3, 'Fast As a Shark', 'medium')
(4, 'Restless and Wild', 'medium')
(5, 'Princess of the Dawn', 'long')
(6, 'Put The Finger On You', 'medium')
(7, "Let's Get It Up", 'medium')
(8, 'Inject The Venom', 'medium')
(9, 'Snowballed', 'medium')
(10, 'Evil Walks', 'medium')


In [57]:
# Aufgabe 1.1: Sortiere (ORDER BY) die Genres so, dass zuerst die Begriffe ohne und dann mit Leerzeichen angezeigt werden (LIKE benutzen).
# Tip: mit || können zwei Strings verbunden werden, '_' wird am Schluss des Alphabets einsortiert.

stmt = """
SELECT *
    FROM Genres
    ;
"""

cur = execute_query(connection, stmt)
print_results(cur)


['GenreId', 'Name']
(1, 'Rock')
(2, 'Jazz')
(3, 'Metal')
(4, 'Alternative & Punk')
(5, 'Rock And Roll')
(6, 'Blues')
(7, 'Latin')
(8, 'Reggae')
(9, 'Pop')
(10, 'Soundtrack')


## 2. VIEW

Eine VIEW enthält die Resultate einer gespeicherten Query, also eines SELECT-Statements. VIEWS werden benutzt,
1. als Abstraktions-Layer über Tabellen. Views können nur bestimmte Spalten einer Tabelle anzeigen, ohne dass das Datenbankmodell verändert werden muss.
2. als Methode, um komplexe Queries mit JOINS einfacher verfügbar zu machen. 
In `sqlite` können VIEWS nur gelesen werden (`SELECT`). `INSERT`, `UPDATE` und `DELETE` funktionieren nicht.

### Syntax
``` sql
CREATE [TEMP] VIEW [IF NOT EXISTS] view_name[(column-name-list)]
AS 
   select-statement;
```

* `TEMP` erzeugt eine temporäre View, die nach beenden der Session wieder glöscht wird. Ansonsten ist die View für immer Verfügbar.
* `column-name-list` erlaubt es eigene Namen für die Spalten einer View zu definieren. Ein Beispiel findet sich hier: https://www.sqlitetutorial.net/sqlite-create-view/


In [4]:
# Beispiel 2.1: erstellen wir nun also eine sogenannte View. Diese kann dann wie eine Tabelle verwendet werden.
# Views werden bei jedem Aufruf neu berechnet.
stmt="""
CREATE VIEW IF NOT EXISTS CustomerInvoices
AS
SELECT
    CustomerId,
    strftime('%Y',InvoiceDate) Year,
    SUM( total ) Total
FROM
    invoices
    GROUP BY CustomerId, strftime('%Y',InvoiceDate);
"""
cur = execute_query(connection, stmt)
print(desc_table(connection, "CustomerInvoices"))

CREATE VIEW CustomerInvoices
AS
SELECT
	CustomerId,
	strftime('%Y',InvoiceDate) Year,
	SUM( total ) Total
FROM
	invoices
	GROUP BY CustomerId, strftime('%Y',InvoiceDate)


In [6]:
# Aufgabe 2.1: Wähle ein SELECT-Statement aus ws01 und erstelle eine View daraus. Erstelle einige Abfragen auf dieser View.

## 3. User Defined Functions (UDF)
UDFs sind in Python erstellte Funktionen, welche bei `sqlite` registriert werden und dann in einem SQL-Statement verwendet werden können.

``` python
def _myfunc(p1, p2):
	pass

# syntax
connection.create_function(name, num_params, func)

# beispiel
connection.create_function(‘myfunc‘, 2, _myfunc)
```


In [7]:
# Beispiel 3.1: String Manipulation
    
import sqlite3

def _toTitleCase(string):
    return str(string).title()
def _toUpperCase(string):
    return str(string).upper()

connection.create_function("title", 1, _toTitleCase)
connection.create_function("upper", 1, _toUpperCase)

stmt="""SELECT name, title(name), upper(name) FROM genres;"""

cur = execute_query(connection, stmt)
print_results(cur)

['Name', 'title(name)', 'upper(name)']
('Rock', 'Rock', 'ROCK')
('Jazz', 'Jazz', 'JAZZ')
('Metal', 'Metal', 'METAL')
('Alternative & Punk', 'Alternative & Punk', 'ALTERNATIVE & PUNK')
('Rock And Roll', 'Rock And Roll', 'ROCK AND ROLL')
('Blues', 'Blues', 'BLUES')
('Latin', 'Latin', 'LATIN')
('Reggae', 'Reggae', 'REGGAE')
('Pop', 'Pop', 'POP')
('Soundtrack', 'Soundtrack', 'SOUNDTRACK')


In [8]:
# Beispiel 3.2: String Formatierung
def _formatBytes(bytes):
    fmt = "b"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "kb"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "mb"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "gb"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "tb"
    return "%s%s" % (round(bytes,2), fmt)

connection.create_function("formatbytes", 1, _formatBytes)

stmt="""SELECT name, formatbytes(bytes) FROM tracks LIMIT 10;"""
cur = execute_query(connection, stmt)
print_results(cur)

['Name', 'formatbytes(bytes)']
('For Those About To Rock (We Salute You)', '10.65mb')
('Balls to the Wall', '5.26mb')
('Fast As a Shark', '3.81mb')
('Restless and Wild', '4.13mb')
('Princess of the Dawn', '6.0mb')
('Put The Finger On You', '6.4mb')
("Let's Get It Up", '7.28mb')
('Inject The Venom', '6.54mb')
('Snowballed', '6.29mb')
('Evil Walks', '8.21mb')


In [9]:
# Beispiel 3.3: existierende Python-Funktionen registrieren.
import math;
connection.create_function("log", 1, math.log10)
stmt="""SELECT log(milliseconds) FROM tracks LIMIT 10;""" # dieses Beispiel ist nur illustrativ und macht keinen wirklichen Sinn.
cur = execute_query(connection, stmt)
print_results(cur)

['log(milliseconds)']
(5.536203539606202,)
(5.534739185524943,)
(5.3628950846379295,)
(5.401488424819678,)
(5.574515091709049,)
(5.313154054869936,)
(5.36907849469925,)
(5.323940648389206,)
(5.307714200040391,)
(5.420775674991212,)


In [10]:
# Aufgabe 3.1: Implementiere die 'short', 'medium' und 'long'-Funtionalität aus Beispiel 1.1 als UDF


# Lösungen

In [11]:
# Aufgabe 1.1: Sortiere (ORDER BY) die Genres so, dass zuerst die Begriffe ohne und dann mit Leerzeichen angezeigt werden (LIKE benutzen).
# Tip: mit || können zwei Strings verbunden werden, '_' wird am Schluss des Alphabets einsortiert.

stmt = """
SELECT *
    FROM Genres
ORDER BY CASE 
    WHEN name LIKE '% %' THEN ('_' || name)
    ELSE name
    END
    ;
"""

cur = execute_query(connection, stmt)
print_results(cur)


['GenreId', 'Name']
(23, 'Alternative')
(6, 'Blues')
(24, 'Classical')
(22, 'Comedy')
(21, 'Drama')
(15, 'Electronica/Dance')
(2, 'Jazz')
(7, 'Latin')
(3, 'Metal')
(25, 'Opera')


In [12]:
# Aufgabe 2.1: keine Lösung, da generisch. Bitte nachfragen bei Verständnisproblemen.

In [13]:
# Aufgabe 3.1: Implementiere die 'short', 'medium' und 'long'-Funtionalität aus Beispiel 1.1 als UDF
def _track_size(milliseconds):
    if milliseconds < 60000:
        return 'short'
    if milliseconds < 300000:
        return 'medium'
    return 'long'

connection.create_function("size", 1, _track_size)

stmt = """SELECT
    trackid, name, size(milliseconds)
FROM
    tracks
LIMIT 10;
"""
cur = execute_query(connection, stmt)
print_results(cur)

['TrackId', 'Name', 'size(milliseconds)']
(1, 'For Those About To Rock (We Salute You)', 'long')
(2, 'Balls to the Wall', 'long')
(3, 'Fast As a Shark', 'medium')
(4, 'Restless and Wild', 'medium')
(5, 'Princess of the Dawn', 'long')
(6, 'Put The Finger On You', 'medium')
(7, "Let's Get It Up", 'medium')
(8, 'Inject The Venom', 'medium')
(9, 'Snowballed', 'medium')
(10, 'Evil Walks', 'medium')
